In [ ]:
import pandas

df = pandas.read_csv("./opensearch_results_merged.csv", low_memory=False)
print(df.shape)
print(df.columns)
print(df.head())
print(df.info())
df.describe()
df.nunique()
print(df.duplicated())
df=df.drop_duplicates()
df=df.dropna(thresh=5)



In [ ]:
print("[*] Shape after cleaning:", df.shape)

print("Dropped rows (%):", (1 - len(df)/306813) * 100)


In [ ]:
missing_ratio = df.isna().mean().sort_values(ascending=False)
print(missing_ratio.head(376)) 

In [ ]:
unique_counts = df.nunique().sort_values()
print(unique_counts.head(100))   
print(unique_counts.tail(100))   

In [ ]:
drop_missing = missing_ratio[missing_ratio > 0.95].index.tolist()
df_reduced = df.drop(columns=drop_missing)

constant_cols = [col for col in df.columns if df[col].nunique() <= 1]
df = df.drop(columns=constant_cols)
print(f"Dropped {len(constant_cols)} constant columns")

unique_counts = df_reduced.nunique()
drop_unique = unique_counts[unique_counts <=1 ].index.tolist()
df_reduced = df_reduced.drop(columns=drop_unique)

print("[*] Reduced dataset shape:",df_reduced.shape)


In [ ]:
empty_rows = df_reduced[df_reduced.isna().any(axis=1)]
print(len(empty_rows))
df_reduced = df_reduced.dropna(how='all')

dup_count = df_reduced.duplicated().sum()
print(dup_count)

df_reduced = df_reduced.drop_duplicates()

threshold=0.9 #if 90% columns of the rows are NaN
row_missing_ratio = df_reduced.isna().mean(axis=1)
df_reduced = df_reduced[row_missing_ratio < threshold]


In [ ]:
non_missing_ratio = 1 - df_reduced.isna().mean()
print(non_missing_ratio.sort_values(ascending=False).head(20))

non_null_counts = df.notna().sum(axis=1)
threshold = 20   # number of non-null values to consider a row 'dense'

dense_rows = df[non_null_counts >= threshold]
sparse_rows = df[non_null_counts < threshold]

print(f"[*] Dense rows (>= {threshold} non-null values): {len(dense_rows)}")
print(f"[*] Sparse rows (< {threshold} non-null values): {len(sparse_rows)}")


In [ ]:
import matplotlib.pyplot as plt

raw_non_missing = df_reduced.notna().sum(axis=1)
plt.hist(raw_non_missing, bins=50, edgecolor="k")
plt.title("Non-missing values per row")
plt.xlabel("count of present columns")
plt.ylabel("row frequency")
plt.show()

In [ ]:
df.to_csv("opensearch_reduced.csv", index=False)
dense_rows.to_csv("opensearch_dense.csv", index=False)
sparse_rows.to_csv("opensearch_sparse.csv", index=False)

print("[+] Cleaned dataset saved:")
print("    - opensearch_reduced.csv (all cleaned data)")
print("    - opensearch_dense.csv   (rows with enough information)")
print("    - opensearch_sparse.csv  (rows with little information)")

df.head()